# Vanilla Head Pose Tracking
This notebook contains the instructions for the first vanilla version of our head pose tracking algorithm. It simply loads the fast.ai pre-trained ImageNet model and trains it on the BIWI Kinect Headpose Dataset. 

Source: fast.ai's lesson3-head-pose.ipynb notebook

## Installation
The following pip commands need to be run the very first time you run the notebook. They install the necessary packages for running the code. 

Source: The instructions come from [here](https://gist.github.com/gilrosenthal/58e9b4f9d562d000d07d7cf0e5dbd840) and [here](https://forums.fast.ai/t/importerror-cannot-import-name-as-tensor/25295/4).

In [0]:
# !pip install fastai
# !apt-get -qq install -y libsm6 libxext6 && pip install -q -U opencv-python
# !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.0.post4-{platform}-linux_x86_64.whl torchvision
# !pip install Pillow==4.0.0
# !pip install image
# !pip install torchtext==0.2.3

## Import
The following cells import the necessary modules for running a notebook with fast.ai. They need to be run every time the notebook starts.

In [0]:
import cv2
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
import torch

In [0]:
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [0]:
from fastai.vision import *